In [1]:
import numpy as np
from astropy.io import fits
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from scipy.stats import gaussian_kde
import scipy.ndimage as ndimage
from matplotlib import cm
from matplotlib.colors import Normalize 
from scipy.interpolate import interpn
import os

In [2]:
indir_flats = '../output_fits/{0}_run/'

In [3]:
filters = ['FELH0600','BG40','SDSSg']
date_dict = {'SDSSg':'20220608','SDSSr':'','empty':'','BG40':'20220608','FELH0600':'20220608'}

In [4]:
indir_dict = {}
for filter in filters:
    indir_dict[filter] = '../output_fits/{0}_run/'.format(date_dict[filter])
print(indir_dict)

filter_files = {}
for filter in filters:
    filter_files[filter] = [f for f in os.listdir(indir_flats.format(date_dict[filter])) if 'master_flat_median_{0}_{1}'.format(date_dict[filter],filter) in f and '_renormalised.fits' in f and 'ratio' not in f][0]
print(filter_files)

{'FELH0600': '../output_fits/20220608_run/', 'BG40': '../output_fits/20220608_run/', 'SDSSg': '../output_fits/20220608_run/'}
{'FELH0600': 'master_flat_median_20220608_FELH0600_87_167_renormalised.fits', 'BG40': 'master_flat_median_20220608_BG40_66_86_renormalised.fits', 'SDSSg': 'master_flat_median_20220608_SDSSg_45_65_renormalised.fits'}


In [5]:
filter1 = 'FELH0600' # Redder filter 
filter2 = 'BG40' # Bluer filter 

In [16]:
if indir_dict[filter1]!=indir_dict[filter2]:
    print('Attention: saving ratio fits files in {0} directory'.format(filter1))
output_dir_fits = indir_dict[filter1]

In [6]:
fred = fits.open(indir_dict[filter1]+filter_files[filter1])
fblue = fits.open(indir_dict[filter2]+filter_files[filter2])

In [8]:
num_segments = 16
shape0 = fred[1].data.shape
hprescan = 3
hoverscan = 64
voverscan = 48

# Ratio of median filters (no smoothing) 

In [9]:
for seg in range(1,num_segments+1):
    print(fred[seg].header['EXTNAME'], fblue[seg].header['EXTNAME'])
    assert (fred[seg].header['EXTNAME']==fblue[seg].header['EXTNAME'])

Segment00 Segment00
Segment01 Segment01
Segment02 Segment02
Segment03 Segment03
Segment04 Segment04
Segment05 Segment05
Segment06 Segment06
Segment07 Segment07
Segment17 Segment17
Segment16 Segment16
Segment15 Segment15
Segment14 Segment14
Segment13 Segment13
Segment12 Segment12
Segment11 Segment11
Segment10 Segment10


In [10]:
new_hdul = fits.HDUList()
hdu1 = fits.PrimaryHDU()
hdu1.header = fred[0].header
hdu1.header['PROCESS'] = 'Ratio-Smooth-Master-Flat'
hdu1.data = fred[0].data
new_hdul.append(hdu1)

for seg in range(1,num_segments+1):
    print(seg)
    hdu2 = fits.ImageHDU()
    red_seg = fred[seg].data
    blue_seg = fblue[seg].data
    
    seg_ratio = np.zeros(red_seg.shape)
    
    seg_ratio[shape0[0]-voverscan:,:] = red_seg[shape0[0]-voverscan:,:]
    seg_ratio[:,:hprescan] = red_seg[:,:hprescan]
    seg_ratio[:,shape0[1]-hoverscan:] = red_seg[:,shape0[1]-hoverscan:]
    
    data_red = red_seg[:shape0[0]-voverscan,hprescan:shape0[1]-hoverscan]
    data_blue = blue_seg[:shape0[0]-voverscan,hprescan:shape0[1]-hoverscan]
    ratio = data_red/data_blue
    
    seg_ratio[:shape0[0]-voverscan,hprescan:shape0[1]-hoverscan] = ratio
    
    hdu2.header = fred[seg].header
    hdu2.data = seg_ratio
    new_hdul.append(hdu2)
    print('----------')
    
new_hdul.writeto(output_dir_fits+'ratio_master_flat_median_{0}_{1}_{2}_{3}_renormalised.fits'.format(filter1,date_dict[filter1],filter2,date_dict[filter2]), overwrite=True)

1
----------
2
----------
3
----------
4
----------
5
----------
6
----------
7
----------
8
----------
9
----------
10
----------
11
----------
12
----------
13
----------
14
----------
15
----------
16
----------


# Ratio of master flats with smooth component removed (high frequency master flats) 

In [10]:
filter_files_hf = {}
for filter in filters:
    filter_files_hf[filter] = [f for f in os.listdir(indir_flats.format(date_dict[filter])) if 'master_flat_median_{0}_{1}'.format(date_dict[filter],filter) in f and '_renormalised_high_frequency.fits' in f and 'ratio' not in f][0]
print(filter_files_hf)

{'FELH0600': 'master_flat_median_20220608_FELH0600_87_167_renormalised_high_frequency.fits', 'BG40': 'master_flat_median_20220608_BG40_66_86_renormalised_high_frequency.fits', 'SDSSg': 'master_flat_median_20220608_SDSSg_45_65_renormalised_high_frequency.fits'}


In [13]:
fredhf = fits.open(indir_dict[filter1]+filter_files_hf[filter1])
fbluehf = fits.open(indir_dict[filter2]+filter_files_hf[filter2])

In [14]:
for seg in range(1,num_segments+1):
    print(fredhf[seg].header['EXTNAME'], fbluehf[seg].header['EXTNAME'])
    assert (fredhf[seg].header['EXTNAME']==fbluehf[seg].header['EXTNAME'])

Segment00 Segment00
Segment01 Segment01
Segment02 Segment02
Segment03 Segment03
Segment04 Segment04
Segment05 Segment05
Segment06 Segment06
Segment07 Segment07
Segment17 Segment17
Segment16 Segment16
Segment15 Segment15
Segment14 Segment14
Segment13 Segment13
Segment12 Segment12
Segment11 Segment11
Segment10 Segment10


In [13]:
new_hdul = fits.HDUList()
hdu1 = fits.PrimaryHDU()
hdu1.header = fredhf[0].header
hdu1.header['PROCESS'] = 'Ratio-Smooth-Master-Flat'
hdu1.data = fredhf[0].data
new_hdul.append(hdu1)

for seg in range(1,num_segments+1):
    print(seg)
    hdu2 = fits.ImageHDU()
    red_seg = fredhf[seg].data
    blue_seg = fbluehf[seg].data
    
    seg_ratio = np.zeros(red_seg.shape)
    
    seg_ratio[shape0[0]-voverscan:,:] = red_seg[shape0[0]-voverscan:,:]
    seg_ratio[:,:hprescan] = red_seg[:,:hprescan]
    seg_ratio[:,shape0[1]-hoverscan:] = red_seg[:,shape0[1]-hoverscan:]
    
    data_red = red_seg[:shape0[0]-voverscan,hprescan:shape0[1]-hoverscan]
    data_blue = blue_seg[:shape0[0]-voverscan,hprescan:shape0[1]-hoverscan]
    ratio = data_red/data_blue
    
    seg_ratio[:shape0[0]-voverscan,hprescan:shape0[1]-hoverscan] = ratio
    
    hdu2.header = fredhf[seg].header
    hdu2.data = seg_ratio
    new_hdul.append(hdu2)
    print('----------')
    
new_hdul.writeto(output_dir_fits+'ratio_master_flat_median_{0}_{1}_{2}_{3}_renormalised_high_frequency.fits'.format(filter1,date_dict[filter1],filter2,date_dict[filter2]), overwrite=True)

1
----------
2
----------
3
----------
4
----------
5
----------
6
----------
7
----------
8
----------
9
----------
10
----------
11
----------
12
----------
13
----------
14
----------
15
----------
16
----------
